In [1]:
def get_tree_percent(x):
    return x['geometry_tree'].intersection(x['geometry']).area/x['geometry'].area
#tract_taz['tract_pct'] = tract_taz[['geometry','geometry_taz']].apply(get_percent, axis = 1)


In [15]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import gdal
import rasterio
import fiona
import rasterio.mask

In [53]:
from rastertodataframe import raster_to_dataframe

In [17]:
boundaryDir = r'data\boundaries'
treeDir = r'data\trees'

In [9]:
muni = gpd.read_file(os.path.join(boundaryDir,'MUNICIPAL_BOUNDARIES.shp'))

In [18]:
delmar = gpd.read_file(os.path.join(boundaryDir,"DEL_MAR.shp"))
#parcel = gpd.read_file(os.path.join(boundaryDir,'PARCELS.shp'))
#tracts = gpd.read_file(os.path.join(boundaryDir,'CENSUS_TRACTS_2010.shp'))
#muni = gpd.read_file(os.path.join(boundaryDir,'MUNICIPAL_BOUNDARIES.shp'))

In [19]:
delmar.crs

{'init': 'epsg:4326'}

In [11]:
landuse = gpd.read_file(os.path.join(boundaryDir,'LAND_USE_2013.shp'))


In [31]:
dmtrees = gpd.read_file(os.path.join(treeDir,'delmar_trees.shp'))

In [36]:
tree_mask = dmtrees.geometry.unary_union


## Process Raster Data

In [21]:
# rasterio.open??
dmt = rasterio.open(os.path.join('data','images','delMar_2014_georeferenced.png'), crs = 'epsg:4326')

In [27]:
#crop del mar from raster
shapes = [list(delmar['geometry'])]
#with rasterio.open(os.path.join('data','images','delMar_2014_georeferenced.png'), crs = 'epsg:4326') as src:
out_image, out_transform = rasterio.mask.mask(dmt, shapes, crop=True)
out_meta = src.meta

In [30]:
#save del mar raster
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open(os.path.join('data','images',"delmar_masked.tif"), "w", **out_meta) as dest:
    dest.write(out_image)

In [37]:
#tree mask
dmt = rasterio.open(os.path.join('data','images','delmar_masked.tif'), crs = 'epsg:4326')

In [64]:
out_image_trees, out_transform_trees = rasterio.mask.mask(dmt, tree_mask, crop=False, invert = True)
out_meta_trees = src.meta
out_image_not_trees, out_transform_not_trees = rasterio.mask.mask(dmt, tree_mask, crop=False, invert = False)
out_meta_not_trees = src.meta


In [65]:
out_meta_trees.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open(os.path.join('data','images',"delmar_masked_trees.tif"), "w", **out_meta_trees) as dest:
    dest.write(out_image_trees)

In [66]:
out_meta_not_trees.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open(os.path.join('data','images',"delmar_masked_not_trees.tif"), "w", **out_meta_not_trees) as dest:
    dest.write(out_image_not_trees)

# Load Training Data

In [71]:
dmtgrtree = raster_to_dataframe(os.path.join('data','images','delmar_masked_trees.tif'))
dmtgrtree['tree'] = 1

In [72]:
dmtgrntree = raster_to_dataframe(os.path.join('data','images','delmar_masked_not_trees.tif'))
dmtgrntree['tree'] = 0

In [73]:
tree_training = pd.concat([dmtgrtree,dmtgrntree], axis = 0)

In [74]:
tree_training.corr()

,Band_1,Band_2,Band_3,Band_4,tree
Band_1,1.000000,0.996181,0.991689,0.922184,0.400241
Band_2,0.996181,1.000000,0.995577,0.925592,0.398067
Band_3,0.991689,0.995577,1.000000,0.898449,0.393103
Band_4,0.922184,0.925592,0.898449,1.000000,0.396207
tree,0.400241,0.398067,0.393103,0.396207,1.000000
